In [1]:
import matplotlib as mpl
mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import cv2
import open3d
from intrinsics import Intrinsics
from utils import pcd_to_depth_map, pcd_to_volume
import math
import scipy
from scipy.spatial.transform import Rotation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
fps = 30
test_name = "test1"
scale = 0.00025
paths = glob.glob(os.path.join("techmed_test", test_name, "depth", "*.png"))
filename = paths[0].split("\\")[-1]

depth = np.asanyarray(open3d.io.read_image(os.path.join("techmed_test", test_name, "depth", filename))) * scale
depth = open3d.geometry.Image(depth.astype(np.float32))

rgb = cv2.imread(os.path.join("techmed_test", test_name, "rgb", filename))

In [3]:
figs, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
axs[0].axis('off')
axs[1].imshow(depth, cmap="gray")
axs[1].axis('off')
plt.show()

In [4]:
points = []

def mouse_event(event):
    global points
    global rgb_copy

    points.append([int(event.xdata), int(event.ydata)])

    if len(points) == 3:
        plt.close()

fig = plt.figure(figsize=(10, 10))
cid = fig.canvas.mpl_connect('button_press_event', mouse_event)
plt.imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
plt.show()

rgb_copy = rgb.copy()
for p in points:
    marked_image = cv2.circle(rgb_copy, p, 3, (0, 255, 0), 2)

fig = plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(rgb_copy, cv2.COLOR_BGR2RGB))
plt.show()

In [58]:
npdepth = np.asarray(depth)
point_mask = np.zeros(npdepth.shape)

for p in points:
    point_mask[p[1], p[0]] = npdepth[p[1], p[0]]

markers_o3d = open3d.geometry.Image(point_mask.astype(np.float32))

m_pcd = open3d.geometry.PointCloud.create_from_depth_image(markers_o3d, intrinsic=intr.get_intrinsics())
m_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
m_pcd.paint_uniform_color([1.0, 0, 0])

points3d = np.asarray(m_pcd.points).copy()

u_1 = points3d[1] - points3d[2]
u_2 = points3d[0] - points3d[2]

n = np.cross(u_1, u_2)
R = Rotation.align_vectors([0, 1, 0], n)[0].as_matrix()

In [63]:
m_pcd.translate(-points3d[2])
m_pcd.rotate(R, center=(0, 0, 0))

PointCloud with 3 points.

In [90]:
def apply_bounding_box(pcd, roi_points):
    p1 = roi_points[np.argmin(roi_points[:, 0])]
    p2 = roi_points[np.argmin(roi_points[:, 2])]
    
    p1 = p1 - np.array([0, 0, .3])
    p2 = p2 - np.array([-.3, -1, .3])
    
    points_to_filter = np.asarray(pcd.points)
    
    x_bound = (points_to_filter[:, 0] > p1[0]) & (points_to_filter[:, 0] < p2[0])
    z_bound = (points_to_filter[:, 2] < p1[2]) & (points_to_filter[:, 2] > p2[2])
    y_bound = (points_to_filter[:, 1] > p1[1]) & (points_to_filter[:, 1] < p2[1])
    
    return pcd.select_by_index(np.where(x_bound & z_bound & y_bound)[0])

pcd_filtered = apply_bounding_box(pcd_body, np.array(m_pcd.points))

In [91]:
open3d.visualization.draw_geometries([pcd_filtered, axes])

<h3>Video</h3>

In [98]:
ref_with_body = np.asanyarray(open3d.io.read_image(paths[400])) * scale
ref_with_body = open3d.geometry.Image(ref_with_body.astype(np.float32))
ref_with_body_pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=ref_with_body, intrinsic=intr.get_intrinsics())
ref_with_body_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
ref_with_body_pcd.translate(-points3d[2])
ref_with_body_pcd.rotate(R, center=(0, 0, 0))

ref_with_only_body_pcd = apply_bounding_box(ref_with_body_pcd, np.array(m_pcd.points))

In [99]:
open3d.visualization.draw_geometries([ref_with_only_body_pcd])

In [30]:
ref_volume

0.2072722310043096

In [29]:
points3d

array([[ 7.42408029e-01,  0.00000000e+00, -2.02774866e+00],
       [ 5.95512388e-01, -8.67361738e-19,  1.62712066e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [102]:
vis = open3d.visualization.Visualizer()
vis.create_window()

geometry = open3d.geometry.PointCloud()
vis.add_geometry(geometry)

fps=30
img=None

diff = []
for i, path in enumerate(paths[300:]):
    depth = np.asanyarray(open3d.io.read_image(path)) * scale
    depth = open3d.geometry.Image(depth.astype(np.float32))
    
    pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=depth, intrinsic=intr.get_intrinsics())
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    pcd.translate(-points3d[2])
    pcd.rotate(R, center=(0, 0, 0))

    pcd = apply_bounding_box(pcd, np.asarray(m_pcd.points))
    
    dists = pcd.compute_point_cloud_distance(ref_with_only_body_pcd)
    m_dists = np.mean(np.asarray(dists))
    diff.append(m_dists)

    geometry.points = pcd.points
        
    if i == 0:
        vis.add_geometry(geometry)

    vis.update_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


C:\Users\kjwdamme\Anaconda3\envs\realsense\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\kjwdamme\Anaconda3\envs\realsense\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [107]:
fig = plt.figure()
y2 = gaussian_filter1d(np.array(diff[:350]), 3)
plt.plot(np.array(diff[:350]), label='original data')
plt.xlabel("difference")
plt.xlabel("frame")
plt.show()